# PockDock

This notebook demonstrates how a variety of different tools can be glued together into an efficient and flexible workflow using **Crossflow**.

The workflow downloads a protein-ligand complex form the PDB, runs fpocket, then docks the ligand back into the biggest pocket found. Then it calculates the error between the crystal structure coordinates of the ligand and those of each docking pose, before and after least-squares fitting.

The notebook requires you to have versions of **fpocket**, **autodock tools** and **autodock vina** installed on the worker node(s) of your dask cluster, which either:

 - is up and running and identifiable via the file "dask.dat" in the current directory (created when dask-scheduler is started with the `--scheduler_file` option)
 - will be a local cluster created right here, in which case scheduler_file=None

In [ ]:
from crossflow import filehandling, kernels, clients
import sys
from urllib.request import urlretrieve
import numpy as np
import mdtraj as mdt

Create a crossflow client, connected to a pool of workers (see introductory notes):

In [ ]:
scheduler_file = None
client = clients.Client(scheduler_file=scheduler_file)
client.client

Make the SubprocessKernels for **fpocket** and **Vina**, and FunctionKernels for other tasks:

In [ ]:
# The fpocket kernel:
fpocket = kernels.SubprocessKernel('fpocket -f x.pdb')
fpocket.set_inputs(['x.pdb'])
fpocket.set_outputs(['x_out/x_out.pdb'])

In [ ]:
# The vina kernel:
vina = kernels.SubprocessKernel('vina --receptor r.pdbqt --ligand l.pdbqt --out out.pdbqt --log dock.log'
                                 ' --center_x {xc} --center_y {yc} --center_z {zc}'
                                 ' --size_x {sx} --size_y {sy} --size_z {sz}')
vina.set_inputs(['r.pdbqt', 'l.pdbqt', 'xc', 'yc', 'zc', 'sx', 'sy', 'sz'])
vina.set_outputs(['out.pdbqt', 'dock.log'])

In [ ]:
# AutoDock Tool based kernels to prepare receptor and ligand for docking:
prep_receptor = kernels.SubprocessKernel('adt prepare_receptor4.py -r x.pdb -o x.pdbqt')
prep_receptor.set_inputs(['x.pdb'])
prep_receptor.set_outputs(['x.pdbqt'])

prep_ligand = kernels.SubprocessKernel('adt prepare_ligand4.py -l x.pdb -o x.pdbqt')
prep_ligand.set_inputs(['x.pdb'])
prep_ligand.set_outputs(['x.pdbqt'])

In [ ]:

def _download_and_split(pdb_code, ligand_residue_name):
    '''
    A function to download a pdb file, and split into receptor and ligand
    
    Args:
        pdb_code (str): 4-letter PDB code
        ligand_residue_name (str): 3-letter residue name
        
    Returns:
        mdt.trajectory: the receptor (protein atoms only)
        mdt.trajectory: the ligand
    '''
    pdb_file = pdb_code + '.pdb'
    path = urlretrieve('http://files.rcsb.org/download/' + pdb_file, pdb_file)
    hydrated_complex = mdt.load(pdb_file)
    receptor_atoms = hydrated_complex.topology.select('protein and chainid 0')
    found = False
    for chain in hydrated_complex.topology.chains:
        for r in chain.residues:
            if r.name == ligand_residue_name and not found:
                cid = chain.index
                found = True

    ligand_atoms = hydrated_complex.topology.select('resname {} and chainid {}'.format(ligand_residue_name, cid))
    receptor = mdt.load(pdb_file, atom_indices=receptor_atoms)
    ligand = mdt.load(pdb_file, atom_indices=ligand_atoms)
    return receptor, ligand
# Now make a FunctionKernel for it:
download_and_split = kernels.FunctionKernel(_download_and_split)
download_and_split.set_inputs(['pdb_code', 'ligand_residue_name'])
download_and_split.set_outputs(['receptor', 'ligand'])

In [ ]:

def _pdbqt2pdb(infile):
    '''
    A Function to convert pdbqt files back to pdb ones
    
    Args:
        infile (str): name of the input file, .pdbqt format
    
    Returns:
        str: name of the .pdb file (always 'tmp.pdb')
    '''
    outfile = 'tmp.pdb'
    fout = open(outfile, 'w')
    with open(infile, 'r') as fin:
        for line in fin:
            if line[1:6] in 'ATOM  MODEL ENDMDL':
                fout.write(line)       
    fout.close()
    return 'tmp.pdb'

# Now make a FunctionKernel for this:
pdbqt2pdb = kernels.FunctionKernel(_pdbqt2pdb)
pdbqt2pdb.set_inputs(['infile'])
pdbqt2pdb.set_outputs(['outfile'])

In [ ]:
def _get_dimensions(pockets):
    '''
    A Function to find the centre and extents of the largest pocket found by fpocket
    
    Args:
        pockets (str): Name of the pdb format file produced by fpocket
        
    Returns:
        (float,) * 6: the pocket centre and extents in x/y/z - in Angstroms
    '''
    buffer = 2.0
    t = mdt.load(pockets)
    site = t.topology.select('resname STP and residue 1') # This should be the largest pocket
    # In the next two lines, the factor of 10 is a conversion from nanometres to Angstroms:
    xc, yc, zc = tuple(10 * (t.xyz[0][site].min(axis=0) + t.xyz[0][site].max(axis=0)) / 2)
    sx, sy, sz = tuple(10 * (t.xyz[0][site].max(axis=0) - t.xyz[0][site].min(axis=0)) + buffer)
    return xc, yc, zc, sx, sy, sz

# Now make a FunctionKernel for this:
get_dimensions = kernels.FunctionKernel(_get_dimensions)
get_dimensions.set_inputs(['pockets'])
get_dimensions.set_outputs(['xc', 'yc', 'zc', 'sx', 'sy', 'sz'])

Now we construct the workflow. For convenience it's split up here into sections.

In [ ]:
pdb_code = '1qy1'
ligand_residue_name = 'PRZ'

In [ ]:
receptor, ligand = client.submit(download_and_split, pdb_code, ligand_residue_name)
print(ligand.result())

In [ ]:
# Run fpocket:
pockets = client.submit(fpocket, receptor)

In [ ]:
# Find the dimensions of the biggest pocket
xc, yc, zc, sx, sy, sz = client.submit(get_dimensions, pockets)

In [ ]:
# Prepare receptor and ligand for docking:
receptor_qt = client.submit(prep_receptor, receptor)
ligand_qt = client.submit(prep_ligand, ligand)

In [ ]:
# Run vina:
docks, logfile = client.submit(vina, receptor_qt, ligand_qt, xc, yc, zc, sx, sy, sz)

In [ ]:
# Check the log file:
print(logfile.result().read_text())

In [ ]:
# Convert the docked poses back to PDB format, and calculate unfitted and fitted rmsds 
# using MDTraj:
pdbout = client.submit(pdbqt2pdb, docks)
docktraj = mdt.load(pdbout.result())
rmsd = mdt.rmsd(docktraj, ligand.result()) * 10.0 # nm -> angstroms
dxyz = docktraj.xyz - ligand.result().xyz
msd = (dxyz * dxyz).sum(axis=2).mean(axis=1)
unfitted_rmsd = np.sqrt(msd) * 10.0
print('Pose Fitted   Unfitted')
print('      rmsd      rmsd')
for mode in range(len(docktraj)):
    print('{:3d}   {:5.3f}    {:6.3f}'.format(mode+1, rmsd[mode], unfitted_rmsd[mode]))